In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.colors
import datetime as dt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm
#from ipywidgets import interact_manual, interact

from ipywidgets import HBox, VBox, Checkbox

In [35]:
covid_cases_age_group = pd.read_csv("../data/covid_data/CovidFaelle_Altersgruppe.csv", names=["time","agegroupID","agegroup","state","stateID","num_residents","sex","infected","cured","dead"], delimiter =";",skiprows= 1)
covid_cases_timeline = pd.read_csv("../data/covid_data/CovidFaelle_Timeline_edited_2.csv", names=["Time","state","stateID","num_residents","infected","infected_sum","num_infected_last_7_days","incident_last_7_days","daily_death_count","dead_sum","daily_healed","healed_sum"], delimiter =";",skiprows= 1)
covid_cases_timeline_groupby_disctrict = pd.read_csv("../data/covid_data/CovidFaelle_Timeline_GKZ.csv", names=["time","county","district_id","num_residents","infected","infected_sum","num_infected_last_7_days","incident_last_7_daysle","dead"], delimiter =";",skiprows= 1)
covid_test_data = pd.read_csv("../data/covid_data/CovidFallzahlen.csv", names=["date","num_tests","date_2","FZHosp","FZICU","FZHospFree","FZICUFree","stateID","state"], delimiter =";",skiprows= 1)
hospitalization_data = pd.read_csv("../data/covid_data/Hospitalisierung.csv", delimiter =";")
vaccine_data = pd.read_csv("../data/covid_data/COVID19_vaccination_doses_timeline_edited.csv", names=["date","state_id","state_name","vaccine","dose_number","doses_administered_cumulative"], delimiter =";",skiprows= 1)
demographics_data =  pd.read_csv("../data/covid_data/demographic_austria.csv", names=["state","2019","2020","2021","2022"], delimiter =";",skiprows= 1)                                                                                                          

In [20]:
hospitalization = hospitalization_data.copy()

hospitalization = hospitalization.rename(columns={'Meldedatum': 'date', 'BundeslandID': 'stateID','NormalBettenBelCovid19': 'occupied_normal_beds','IntensivBettenBelCovid19': 'occupied_intensive_care_beds'})

hospitalization = hospitalization.drop(columns=['Bundesland', 'IntensivBettenKapGes', 'IntensivBettenBelNichtCovid19','IntensivBettenFrei','TestGesamt'])


hospitalization['date'] = pd.to_datetime(hospitalization['date'])
hospitalization.set_index('date', inplace=True)

mask = hospitalization['stateID'] != 10
hospitalization = hospitalization.loc[mask]

# dates = pd.date_range('2020-01-01', '2021-01-23', freq='D')
# hospitalization_trick = hospitalization_trick.reindex(dates, fill_value=0)
# hospitalization = pd.concat([hospitalization_added, hospitalization])

hospitalization

,stateID,occupied_normal_beds,occupied_intensive_care_beds
date,,,
2021-01-24,1,38,7
2021-01-24,2,72,9
2021-01-24,3,359,53
2021-01-24,4,240,38
2021-01-24,5,97,22
...,...,...,...
2022-11-11,5,95,3
2022-11-11,6,104,9
2022-11-11,7,47,3


In [68]:
vaccine = vaccine_data.copy()
vaccine = vaccine.drop(columns=['state_name', 'vaccine', 'dose_number'])

mask_10 = vaccine['state_id'] != 10
mask_0 = vaccine['state_id'] != 0
vaccine = vaccine.loc[mask_10]
vaccine = vaccine.loc[mask_0]


vaccine['date'] = pd.to_datetime(vaccine['date'])
vaccine = vaccine[(vaccine['date'] > '2020-01-01') & (vaccine['date'] <= '2022-11-11')]
vaccine.set_index('date', inplace=True)

vaccine = vaccine.groupby('state_id')['doses_administered_cumulative'].resample('D').sum().to_frame()

vaccine = vaccine.reset_index()
vaccine = vaccine.sort_values(by=['date', 'state_id'])
vaccine.set_index('date', inplace=True)

vaccine


,state_id,doses_administered_cumulative
date,,
2020-12-27,1,88
2020-12-27,2,0
2020-12-27,3,188
2020-12-27,4,15
2020-12-27,5,82
...,...,...
2022-11-10,5,1164088
2022-11-10,6,2808845
2022-11-10,7,1621812


In [62]:
daily_death_cop = covid_cases_timeline.copy()

daily_death_cop = daily_death_cop.rename(columns={'Time': 'date'})
daily_death = daily_death_cop.loc[:, ['date','daily_death_count', 'stateID']]


mask = daily_death['stateID'] != 10
daily_death = daily_death.loc[mask]



daily_death['date'] = pd.to_datetime(daily_death['date'], infer_datetime_format=True)
daily_death.set_index('date', inplace=True)

daily_death = daily_death.groupby('stateID')['daily_death_count'].resample('D').sum().to_frame()
daily_death = daily_death.reset_index()
daily_death = daily_death.sort_values(by=['date', 'stateID'])
daily_death.set_index('date', inplace=True)

daily_death['stateID'] = daily_death['stateID'].astype('int')

daily_death

C:\Users\felix\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '13.01.2020' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\felix\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '14.01.2020' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\felix\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '15.01.2020' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\felix\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '16.01.2020' in DD/MM/YYYY format. Provide format or specif

,stateID,daily_death_count
date,,
2020-01-03,1,0.0
2020-01-03,2,0.0
2020-01-03,3,0.0
2020-01-03,4,0.0
2020-01-03,5,0.0
...,...,...
2022-12-10,5,0.0
2022-12-10,6,1.0
2022-12-10,7,0.0


In [69]:
daily_death = daily_death.reset_index()

vaccine = vaccine.reset_index()

hospitalization = hospitalization.reset_index() 

In [70]:
daily_death.to_csv("../data/felix_to_tableau/daily_death_by_state.csv", index=False)
vaccine.to_csv("../data/felix_to_tableau/vaccine_by_state.csv", index=False)
hospitalization.to_csv("../data/felix_to_tableau/hospitalization_by_state.csv", index=False)